# Unit Square

Here we demonstrate how to plot different FEniCSx object for a unit square geometry

First we make the necessary imports

In [ ]:
import dolfinx
import numpy as np
from mpi4py import MPI
import ufl
from fenicsx_plotly import plot

First we create the unit square geometry

In [ ]:
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 3, 3)

Now we can plot the mesh. By default it will plot the mesh in wireframe

In [ ]:
plot(mesh)

We can also turn off wireframe mode

In [ ]:
plot(mesh, wireframe=False)

Now we can try to create a function space and plot the degrees of freedom

In [ ]:
V = dolfinx.fem.FunctionSpace(mesh, ("P",  3))
plot(V)

We can also plot a function in this function space

In [ ]:
p = dolfinx.fem.Function(V)
p.interpolate(lambda x: np.sin(x[0]))
plot(p, scatter=True, wireframe=False, show_grid=True)

We can also create a vector function space

In [ ]:
W = dolfinx.fem.FunctionSpace(mesh, ufl.VectorElement("P", mesh.ufl_cell(), 2))
plot(W)

and create a function in this function space

In [ ]:
u = dolfinx.fem.Function(W)
# Just create create some non-trivial function
x = ufl.SpatialCoordinate(mesh)
expr = dolfinx.fem.Expression(
    ufl.as_vector((1 + x[0], x[1])), W.element.interpolation_points()
)

u.interpolate(expr)

plot(u, size=1)
plot(u, normalize=True, size=1)
plot(u, norm=True, size=1)

We can also plot the different components

In [ ]:
for component in ["magnitude", "x", "y"]:
    plot(u, component=component)

In [ ]:
# Create a facet fuction in order to mark the subdomains
#ffun = df.MeshFunction("size_t", mesh, 2)
#ffun.set_all(0)

#fixed = df.CompiledSubDomain("x[0] < 0.5")
#free = df.CompiledSubDomain("x[0] >= 0.5")
# Mark the first subdomain with value 1

#fixed_marker = 1
#fixed.mark(ffun, fixed_marker)

# Mark the second subdomain with value 2
#free = df.CompiledSubDomain("near(x[0], 1) && on_boundary")
#free_marker = 2
#free.mark(ffun, free_marker)

#plot(ffun)

In [ ]:
#V = df.VectorFunctionSpace(mesh, "CG",  2)
#left = df.CompiledSubDomain("near(x[0], 0) && on_boundary")
#bc = df.DirichletBC(V, df.Constant((0.0, 0.0)), left)
#plot(bc, wireframe=True)